# Huấn luyện có giám sát với SFTTrainer

Bài học này se dạy bạn các huấn luyện mô hình `HuggingFaceTB/SmolLM2-135M` bằng `SFTTrainer` trong thư viện `trl`.  Các cell trong notebook này sẽ chạy và huấn luyện mô hình. Bạn có thể chọn độ khó bằng cách thử nghiệm với các bộ dữ liệu khác nhau.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bài tập: Fine-Tuning SmolLM2 với SFTTrainer</h2>
    <p>Chọn một bộ dự liệu từ Hugging Face hub và huấn luyện một mô hình trên bộ dữ liệu đó. </p> 
    <p><b>Các bài tập</b></p>
    <p>🐢 Sử dụng bộ dữ liệu `HuggingFaceTB/smoltalk`</p>
    <p>🐕 Thử nghiệm với bộ dữ liệu `bigcode/the-stack-smol` và huấn luyện một mô hình sinh code trên tập con cụ thể `data/python`.</p>
    <p>🦁 Chọn một bộ dữ liệu liên quan đến một lĩnh vực mà bạn quan tâm</p>
</div>

In [5]:
# Cài đặt các thư viện cần thiết
!pip install transformers datasets trl huggingface_hub

# Đăng nhập vào Hugging Face
from huggingface_hub import login

login()

# Để thuận tiện, bạn có thể tạo một biến môi trường chứa `token hub` của bạn dưới dạng HF_TOKEN

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 6.3 MB/s eta 0:00:00a 0:00:01


In [20]:
# Các thư viện cần thiết
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Tải mô hình và tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Thiết lập định dạng chat
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Đặt tên cho mô hình huấn luyện để lưu &/ tải lên
finetune_name = "SmolLM2-FT-o1"
finetune_tags = ["smol-course", "module_1"]

# Sinh văn bản với Mô hình gốc

Ở đây chúng ta sẽ thử nghiệm mô hình gốc chưa được huấn luyện trên định dạng chat.

In [21]:
# Kiểm tra mô hình gốc trước khi huấn luyện
prompt = "What is the area, in square units, of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?"

# Định dạng
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Tạo phản hồi từ mô hình
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=4096)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
What is the area, in square units, of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?
What is the area of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?
What is the area of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?
What is the area of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?
What is the area of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?
What is the area of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?
What is the area of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?
What is the area of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?
What is the area of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?
What is the area of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?
What is the area of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?
What is the area of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?
What is the area of a triangle with 

## Chuẩn bị dữ liệu

Chúng ta sẽ tải một bộ dữ liệu mẫu và định dạng nó cho việc huấn luyện. Bộ dữ liệu cần được cấu trúc với các cặp đầu vào - đầu ra, trong đó mỗi đầu vào là một chỉ thị và đầu ra là phản hồi mong đợi từ mô hình.

**TRL sẽ định dạng các tin nhắn đầu vào dựa trên định dạng chat của mô hình** Chúng cần được biểu diễn dưới dạng danh sách các từ điển với các khóa: `role` và `content`.

**Ví dụ:**
```sh
[
  {"role": "user", "content": "Hello, how are you?"},
  {"role": "assistant", "content": "I'm doing well, thank you! How can I assist you today?",},
]
```

In [9]:
# Tải dữ liệu mẫu
from datasets import load_dataset, Dataset, DatasetDict

ds = load_dataset("GAIR/o1-journey")
ds = ds.remove_columns(['idx', 'answer'])
# Split the dataset
split_dataset = ds['train'].train_test_split(test_size=27, seed=42)

# Create a new DatasetDict with 'train' and 'test' splits
ds = DatasetDict({
    'train': split_dataset['train'],
    'test': split_dataset['test']
})

print(ds)

DatasetDict({
    train: Dataset({
        features: ['question', 'longCOT'],
        num_rows: 300
    })
    test: Dataset({
        features: ['question', 'longCOT'],
        num_rows: 27
    })
})


In [10]:
# TODO: 🦁 Nếu dataset của bạn không ở định dạng mà TRL có thể chuyển đổi thành định dạng chat, bạn sẽ cần xử lý nó.
# Tham khảo [Định dạng Chat](../chat_templates.md)

def process_dataset(sample):
    # TODO: 🐢 Chuyển đổi dữ liệu sang định dạng ChatML
    # Sử dụng `tokenizer.apply_chat_template` để áp dụng định dạng hội thoại
    # 1. Tạo định dạng tin nhắn với vai trò và nội dung
    format_message = [
        {"role": "user", "content": str(sample.get('question', ''))},
        {"role": "assistant", "content": str(sample.get('longCOT', ''))},
    ]
    # 2. Áp dụng định dạng chat cho các mẫu bằng tokenizer
    sample = tokenizer.apply_chat_template(format_message, tokenize=False, add_generation_prompt=True)
    return {'text': sample}


ds = ds.map(process_dataset)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

## Điều chỉnh SFTTrainer

Điều chỉnh `SFTTrainer` với các tham số khác nhau giúp điều khiển quá trình huấn luyện trở nên hiệu quả hơn. Các thông số bao gồm
- Số bước huấn luyện (steps)
- Kích thước batch (batch size)
- Tốc độ học (learning rate)
- Chiến lược đánh giá mô hình (evaluation strategy)

Ngoài ra, còn rất nhiều thông số khác, bạn có thể tham khảo thêm ở [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer)

Điều chỉnh các tham số này dựa trên yêu cầu cụ thể và tài nguyên tính toán của bạn.

In [11]:
ds

DatasetDict({
    train: Dataset({
        features: ['question', 'longCOT', 'text'],
        num_rows: 300
    })
    test: Dataset({
        features: ['question', 'longCOT', 'text'],
        num_rows: 27
    })
})

In [12]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=500,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)


# TODO: 🦁 🐕 căn chỉnh các tham số SFTTrainer với bộ dữ liệu bạn đã chọn.
# Ví dụ, nếu bạn đang sử dụng bộ `bigcode/the-stack-smol`, bạn sẽ cần chọn cột `content`

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


## Huấn luyện Mô hình

Với trainer đã được điều chỉnh, chúng ta có thể tiến hành huấn luyện mô hình. Quá trình huấn luyện sẽ bao gồm
- Lặp qua bộ dữ liệu
- Tính toán loss
- Cập nhật các tham số của mô hình để giảm thiểu loss này.

In [13]:
# # Huấn luyện mô hình
trainer.train()

# Lưu mô hình
trainer.save_model(f"./{finetune_name}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
50,1.013500,1.001641
100,0.857100,0.971714
150,0.848300,0.959724
200,0.703800,0.974492
250,0.657200,1.016198
300,0.675400,1.014580
350,0.548300,1.078962
400,0.466400,1.145712
450,0.437500,1.152163
500,0.377900,1.242481


In [14]:
# Đưa mô hình lên Hugging Face Hub
trainer.push_to_hub(tags=finetune_tags)

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1734321200.d54dcbe24fec.23.0:   0%|          | 0.00/32.3k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/quyettv/SmolLM2-FT-o1/commit/6dc089f78ae278d984b0bcb3997a863a59edbc46', commit_message='End of training', commit_description='', oid='6dc089f78ae278d984b0bcb3997a863a59edbc46', pr_url=None, repo_url=RepoUrl('https://huggingface.co/quyettv/SmolLM2-FT-o1', endpoint='https://huggingface.co', repo_type='model', repo_id='quyettv/SmolLM2-FT-o1'), pr_revision=None, pr_num=None)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bài tập thêm: Sinh văn bản với mô hình vừa được huấn luyện</h2>
    <p>🐕 Sử dụng mô hình đã được huấn luyện để sinh ra phản hồi, giống như với ví dụ ban đầu.</p>
</div>

In [23]:
# Kiểm tra mô hình đã được huấn luyện trên cùng một yêu cầu
# Tải mô hình và tokenizer
model_name = "quyettv/SmolLM2-FT-o1"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)


# Kiểm tra mô hình gốc trước khi huấn luyện
prompt = "What is the area, in square units, of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?"

# Định dạng chat
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Sinh phản hồi từ mô hình
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: sử dụng mô hình đã được huấn luyện để sinh phản hồi, giống như với ví dụ.
outputs = model.generate(**inputs, max_new_tokens=4096)
print("After training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

config.json:   0%|          | 0.00/812 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.59k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

After training:
user
What is the area, in square units, of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?
assistant
Let's explore how to find the area of a triangle with vertices at \( A(1, 1), B(6, 1), C(3, 7)\).

First, we know the area of the triangle is calculated as:

\[
\frac{\sqrt{3}}{2} |AB|^2
\]

Now, let's focus on finding \( A(1, 1) \). We can set up the equation:

\[
A(1, 1) = \sqrt{3} |AB|^2
\]

Simplifying the left side, we get:

\[
|AB| = \sqrt{3} |1|
\]

Let's pause and consider what we know so far. We have found \( |AB| = \sqrt{3} |1| \). Now, let's solve for \( |AB| \).

To find \( |AB| \), we calculate the length of one side:

\[
AB = 1
\]

Now, we need the width of the triangle. It should be the opposite side \( |AB| \):

\[
|AB| = 1 - |AD|
\]

where \( |AD| = 1 + \sqrt{3} \).

Substituting these values into our equation, we get:

\[
|AB| = \sqrt{3} |1| - \sqrt{3} |AD| = \sqrt{3} \times 1 + \sqrt{3} = \sqrt{3} |AD|
\]

Let's pause and review what we've don

## 💐 Chúc mừng bạn. Bạn đã hoàn thành!

Bài tập này đã cung cấp hướng dẫn từng bước để bạn huấn luyện được mô hình `HuggingFaceTB/SmolLM2-135M` sử dụng `SFTTrainer`. Bằng cách làm theo các bước này, bạn có thể điều chỉnh mô hình để thực hiện các tác vụ cụ thể hiệu quả hơn. Nếu bạn muốn tiếp tục làm việc với khóa học này, đây là một số bước bạn có thể thử:

- Thử notebook này ở mức độ khó hơn
- Review PR của học viên khác
- Cải thiện tài liệu khóa học thông qua Issue hoặc PR.